In [1]:
import os
import sys

import cv2
import torch
import shutil
import supervision as sv

sys.path.append('/app/')
from src.worker import process_sam

In [8]:
# Директория с изображениями для разметки
SOURCE_IMAGE_PATH = '/app/data/cars_yolo_s/'

# Директория для сохранения набора данных
SOURCE_DIRECTORY_PATH = '/app/data/dataset_cars_yolo_s/'

SAM_ENCODER_VERSION = "vit_h"

data_yaml_path = '/app/data/cars_yolo_s/dataset.yaml'
subsets = ['train', 'val', 'test'] # 'train', 'val', 'test'
classes = ['car', 'lp']

# Веса модели
SAM_WEIGHTS_PATH = "/root/weights/sam_vit_h_4b8939.pth"

In [9]:
if not os.path.isfile(SAM_WEIGHTS_PATH):
    print("Файл с весами модели отсутствует.")

In [10]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
from segment_anything import sam_model_registry, SamPredictor

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_WEIGHTS_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)

In [13]:
annotations = {}
for subset in subsets:
    dataset = sv.DetectionDataset.from_yolo(
        images_directory_path=f'{SOURCE_IMAGE_PATH}images/{subset}',
        annotations_directory_path=f'{SOURCE_IMAGE_PATH}labels/{subset}',
        data_yaml_path=data_yaml_path
    )
    dataset.classes = classes
    dataset.annotations = {f'{SOURCE_IMAGE_PATH}images/{subset}/' + item[0]:item[1] for item in dataset.annotations.items()}
    annotations.update(process_sam(dataset.annotations, sam_predictor, subset, False, SOURCE_DIRECTORY_PATH))

100%|██████████| 170/170 [04:19<00:00,  1.53s/it]
